In [2]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import aif360.datasets.noisy_dataset as noisy
import noisyfair.algorithms as denoisedfair
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, CompasDataset
# from aif360.metrics import BinaryLabelDatasetMetric
# from aif360.metrics import ClassificationMetric
# from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas

# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import StandardScaler, MaxAbsScaler
# from sklearn.metrics import accuracy_score

# from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import numpy as np


In [6]:
# Get the dataset and split into train and test
dataset = load_preproc_data_adult()

protected_name = 'race'
privileged_groups = [{protected_name: 1}]
unprivileged_groups = [{protected_name: 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle=True)
eta1 = 0.3
eta2 = 0.1
index, noisyfea = noisy.flipping(dataset_train.feature_names, dataset_train.features, protected_name, eta1, eta2)

dataset_noisy = np.copy(dataset_train.features)
dataset_noisy[:,index] = noisyfea

print("index:", index)
print(sum(dataset_noisy[:,index]), sum(dataset_train.features[:,index]), sum(dataset_test.features[:,index]))
print(sum(dataset_test.labels), dataset_test.labels.shape)

Count_flipping: 4479
index: 0
27801.0 29210.0 12552.0
[3396.] (14653, 1)


In [3]:
# zvrg
C = 0
thresh = 0.1
zvrg_theta = denoisedfair.zvrg(dataset_noisy, dataset_train.labels, index, C, thresh)

print(zvrg_theta)

acc, sr = denoisedfair.testing(dataset_noisy, dataset_train.labels, index, zvrg_theta)
print(acc, sr)

acc, sr = denoisedfair.testing(dataset_train.features, dataset_train.labels, index, zvrg_theta)
print(acc, sr)

acc, sr = denoisedfair.testing(dataset_test.features, dataset_test.labels, index, zvrg_theta)
print(acc, sr)

[ 1.30901083 -3.13813223 -1.00342494  0.70108146  1.16623278  1.39113965
  0.90497257  0.41944302 -1.00003484 -0.90223678 -0.63812176  0.1136199
  0.60693325  0.80209268  0.85376458 -1.2549231   1.75613526 -3.46644861]
sr0: 0.22262471537983855 sr1: 0.24528071105312513
0.7287431630056451 0.9076323793419714
sr0: 0.19175591028490604 sr1: 0.24404924760601915
0.7294451431747053 0.7857262915822103
sr0: 0.19380572501173157 sr1: 0.25187669701325666
0.7213539889442435 0.7694468258075152


In [4]:
# denoised fair
tau = 0.8
lam = 0.1
C = 0
denoised_theta = denoisedfair.denoised(dataset_noisy, dataset_train.labels, index, C, tau, eta, lam)
print(denoised_theta)

acc, sr = denoisedfair.testing(dataset_noisy, dataset_train.labels, index, denoised_theta)
print(acc, sr)

acc, sr = denoisedfair.testing(dataset_train.features, dataset_train.labels, index, denoised_theta)
print(acc, sr)

acc, sr = denoisedfair.testing(dataset_test.features, dataset_test.labels, index, denoised_theta)
print(acc, sr)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.7923736287751326
            Iterations: 71
            Function evaluations: 71
            Gradient evaluations: 71
[-0.21619628 -1.05538342  0.35319016  0.76357661  1.01352458  0.95408476
  0.35140674 -0.10515213  0.29476184  0.47618543  0.54809863  0.00898934
  0.14767229  0.14035534  0.29043492  0.0739007   0.65099141 -0.28642564]
sr0: 0.6036017387704409 sr1: 0.6003587882741469
0.47883822282020533 0.9946273340714691
sr0: 0.5968882602545968 sr1: 0.6008207934336526
0.47693702652900055 0.9934547318900507
sr0: 0.6025340215861098 sr1: 0.5991854336367992
0.48604381355353854 0.994442491495342
